### SQLAlchemy pre-configuration

In [ ]:
!pip install snowflake-sqlalchemy ipython-sql

#### Step 1: Set the CONNECTION_KEY to the same name defined in the local SnowSQL configuration file

In [ ]:
import re
from os.path import expanduser
from snowflake.sqlalchemy import URL

USER_PATH = expanduser("~")
# set CONNECTION_KEY to the same name defined in the local SnowSQL configuration file
CONNECTION_KEY = "training-jramizares"

with open(f'{USER_PATH}/.snowsql/config') as f:
    connection = {}
    for line in f:
        buffer = None
        if buffer is not None:
            result = re.search(r"\[connections\.(.*)\]", buffer)
            buffer = None
        else:
            result = re.search(r"\[connections\.(.*)\]", line)
        if result:
            connection_key = result.group(1)
            if connection_key != "example":
                config = {}
                for buffer in f:
                    result = re.search(r"\[connections\.(.*)\]", buffer.strip())
                    if buffer.strip() != "":
                        (key, value) = buffer.strip().replace(" ", "").split("=")
                        config[key] = value
                    else:
                        connection[connection_key] = config
                        break

SNOW_LOCATOR = connection[CONNECTION_KEY]["accountname"]
SNOW_USER = connection[CONNECTION_KEY]["username"]
SNOW_PASSWD = connection[CONNECTION_KEY]["password"]
SNOW_DB = connection[CONNECTION_KEY]["database"]
SNOW_SCHEMA = connection[CONNECTION_KEY]["schema"]
SNOW_WAREHOUSE = connection[CONNECTION_KEY]["warehouse"]
SNOW_ROLE = connection[CONNECTION_KEY]["role"]

%reload_ext sql
%sql snowflake://{SNOW_USER}:{SNOW_PASSWD}@{SNOW_LOCATOR}/{SNOW_DB}?role={SNOW_ROLE}&warehouse={SNOW_WAREHOUSE}

### SRR Resources

- Training
    - Engineering
        - [Iceberg Tables Training for Support](https://snowflakecomputing.atlassian.net/wiki/spaces/CustomerSupport/pages/3028713756/Iceberg+Tables+Training+for+Support)
    - Support
        - [Intro to Iceberg Tables - Key Concepts training](https://snowflake.zoom.us/rec/share/997cOWJYC6rBIuyzs2P02oD8f28M7dzWj0ygDG9NuIhw3uti2EUqq8_h1m9VUQ8l.dpBvyZ9p_YEQVkOx)
            - Passcode: <code>AAPQ&W5$</code>    
- Runbook(s)
    - [Iceberg Support Runbook](https://docs.google.com/document/d/18MjH6n3ypi4VEbs5_wa_Vpxgi0XtVWCLW4_EEe7DBLQ/edit#heading=h.1bu2wjpos1k9)
- Product documentation
    - [Iceberg tables](https://docs.snowflake.com/en/user-guide/tables-iceberg)

### What is Apache Iceberg?

[Apache Iceberg](https://iceberg.apache.org/docs/latest/) is an open-source table format specification developed for huge analytic datasets. 

### What are Iceberg Tables?

Iceberg tables are a new table type designed to support the [Apache Iceberg table specification](https://iceberg.apache.org/spec/) to represent a large collection of slowly-changing files on a distributed file system (AWS S3, Azure Blob, Google Cloud Storage) with performance close to that of native Snowflake tables.

The architecture of an Apache Iceberg table is defined as three distinct layers:
- [Iceberg Catalog](https://iceberg.apache.org/concepts/catalog/#iceberg-catalogs) - Used to manage a collection of tables</li>
- [Metadata](https://iceberg.apache.org/spec/#specification) - Used to manage table states in a catalog through a combination of metadata files (JSON), manifest lists (Avro), and manifest files (Avro)
- Data - Collection of files that represent the data for all tables in the catalog

<div>
<img src="https://iceberg.apache.org/img/iceberg-metadata.png" width="50%"/>
</div>


Snowflake supports Iceberg tables with externally managed catalogs (<strong>unmanaged</strong>) and natively managed catalogs (<strong>managed</strong>).

The following catalog types are supported for unmanaged iceberg tables:
- [AWS Glue data catalog](https://docs.aws.amazon.com/glue/latest/dg/catalog-and-crawler.html)
- Object storage
    - Iceberg
    - Loose parquet 

To create an unmanaged Iceberg table, you will need to configure:
- [EXTERNAL VOLUME](https://docs.snowflake.com/sql-reference/sql/create-external-volume) - Defines the location(s) for the external catalog, metadata, and data
- [CATALOG INTEGRATION](https://docs.snowflake.com/en/sql-reference/sql/create-catalog-integration) - Defines the details of the external catalog

### Exercise 1: Create an unmanaged Iceberg table

#### Step 1: Create an external volume

Update the IAM role Trust policy with the AWS User ARN and External ID

In [ ]:
%%sql
use role support_rl;
create external volume if not exists {SNOW_USER}_ext_vol
    storage_locations = (
        (
            name = 's3_iceberg_example_unamanaged_us_west_glue'
            storage_provider = 's3'
            storage_base_url = 's3://kt-s3-us-west-2/'
            storage_aws_role_arn = 'arn:aws:iam::094271313368:role/kterada_custom_s3_role'
            encryption = (type='aws_sse_s3')
        )
    )
;
desc external volume {SNOW_USER}_ext_vol;

#### Step 2: Create a catalog integration for each supported type

##### Step 2.1 Create an OBJECT STORAGE (ICEBERG) catalog integration

In [ ]:
%%sql

use role support_rl;

create catalog integration if not exists {SNOW_USER}_os_iceberg_cat_int
    catalog_source = object_store
    table_format = iceberg
    enabled = true
;
desc integration {SNOW_USER}_os_iceberg_cat_int;

##### Step 2.2 Create an AWS GLUE catalog integration
Step 2.3 Update the IAM role Trust policy with the AWS User ARN and External ID

In [ ]:
%%sql

use role support_rl;
create catalog integration if not exists {SNOW_USER}_glue_cat_int
    catalog_source = glue
    catalog_namespace = 'kterada_db'
    table_format = iceberg
    glue_aws_role_arn = 'arn:aws:iam::094271313368:role/kterada_custom_glue_role'
    glue_catalog_id = '094271313368'
    glue_region = 'us-west-2'
    enabled = true
;
desc integration {SNOW_USER}_glue_cat_int;

#### Step 3: Create the unmanaged Iceberg tables

##### Step 3.1 Create the schema to store the Iceberg tables

In [ ]:
%%sql

use role support_rl;
create warehouse if not exists {SNOW_USER}_wh warehouse_size='xsmall';
create database if not exists {SNOW_USER}_iceberg_db;
create schema if not exists {SNOW_USER}_iceberg_db.iceberg;

##### Step 3.2 Create Unmanaged Iceberg table with OBJECT STORE (ICEBERG) catalog

In [ ]:
%%sql

use role support_rl;
create iceberg table if not exists {SNOW_USER}_iceberg_db.iceberg.{SNOW_USER}_unmanaged_os_iceberg_web_sales_t
    external_volume = {SNOW_USER}_ext_vol
    catalog = {SNOW_USER}_os_iceberg_cat_int
    metadata_file_path = 'athena/tcpds_sf10tcl/web_sales/metadata/00000-01c90780-5d6a-48c5-bbd4-4c6c7a88eec5.metadata.json'
;

##### Step 3.2 Query the Iceberg table

In [ ]:
%%sql

use role support_rl;
use warehouse {SNOW_USER}_wh;
select
    *
from {SNOW_USER}_iceberg_db.iceberg.{SNOW_USER}_unmanaged_os_iceberg_web_sales_t
where true
limit 100
;

##### Step 3.4 Create Unmanaged Iceberg table with GLUE catalog

In [ ]:
%%sql

use role support_rl;
create iceberg table if not exists {SNOW_USER}_iceberg_db.iceberg.{SNOW_USER}_unmanaged_glue_iceberg_customer_t
    external_volume = {SNOW_USER}_ext_vol
    catalog = {SNOW_USER}_glue_cat_int
    catalog_namespace = 'kterada_db'
    catalog_table_name = 'kt_iceberg_tpcds_sf10tcl_customer'
;

##### Step 3.5 Query the Iceberg table

In [ ]:
%%sql

use role support_rl;
use warehouse {SNOW_USER}_wh;
select
    *
from {SNOW_USER}_iceberg_db.iceberg.{SNOW_USER}_unmanaged_glue_iceberg_customer_t
where true
limit 100
;

#### Step 4: Convert Unmanaged to Managed Iceberg table

Step 4.1 Review the current Iceberg table catalog

In [ ]:
%%sql

use role support_rl;
use warehouse {SNOW_USER}_wh;
select get_ddl('table','{SNOW_USER}_iceberg_db.iceberg.{SNOW_USER}_unmanaged_os_iceberg_web_sales_t');

##### Step 4.2 Convert the unmanaged Iceberg table to managed

In [ ]:
%%sql

alter iceberg table {SNOW_USER}_iceberg_db.iceberg.{SNOW_USER}_unmanaged_os_iceberg_web_sales_t convert to managed base_location='athena/tcpds_sf10tcl/web_sales/';

##### Step 4.3 Validate the conversion to managed

In [ ]:
%%sql

use role support_rl;
use warehouse {SNOW_USER}_wh;
select get_ddl('table','{SNOW_USER}_iceberg_db.iceberg.{SNOW_USER}_unmanaged_os_iceberg_web_sales_t');